In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# set the max columns to none
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'

import statistics
from statistics import mode

## Finalizing Main Dataset

In [3]:
# import dataset hotel, review, and user

# hotel menunggu revisi joane
df_hotel = pd.read_csv('https://raw.githubusercontent.com/zulfauzi92/Hotel_Recomendation_Model_Traveloka/main/Eksplorasi%20Data/Main%20Dataset/DataHotel_ML.csv', index_col=[0])
df_user = pd.read_csv('https://raw.githubusercontent.com/zulfauzi92/Hotel_Recomendation_Model_Traveloka/main/Eksplorasi%20Data/Main%20Dataset/DataUser.csv', index_col=[0])
df_review = pd.read_csv('https://raw.githubusercontent.com/zulfauzi92/Hotel_Recomendation_Model_Traveloka/main/Eksplorasi%20Data/Main%20Dataset/DataReview.csv', index_col=[0])
print(df_hotel.shape)
print(df_user.shape)
print(df_review.shape)

(1311, 383)
(9219, 4)
(9787, 11)


### Hotel Data

In [4]:
# rename some features
df_hotel.rename(columns={'property_id':'hotel_id', 'property_name':'hotel_name'}, inplace=True)

In [48]:
# Non-ML Hotel Feature
hotel_nonML_col = np.append(df_hotel.columns[:8].values,['property_type','hotel_facilities','room_facilities'])

# ML Hotel Feature
hotel_ML_col = np.append(['hotel_id','latitude','longitude','hotel_star_rating'], df_hotel.columns[14:].values)
df_ML_hotel = df_hotel[hotel_ML_col]
df_ML_hotel

,hotel_id,latitude,longitude,hotel_star_rating,site_review_rating,Bike on Rent,Cook Services,Drivers Rest Room,24 Hour Front Desk,ATM / Banking,Access Via Exterior Corridors,Adjoining Rooms,Air / Rail Booking,Air-Conditioning - Central,Airport Transfer Available / Surcharge,Airport Transfer Free,Amphitheatre,Anti-Slip Ramps,Audio - Visual Equipment,Ayurvedic Facilities,Ayurvedic Facilities,Babysitting/ Child Care (surcharge),Badminttion Court,Baggage Room,Ballroom,Banquet Facilities,Bar / Lounge,Barbecue Area,Basketball Court,Beauty Salon - on charge,Billiards,Board Room,Boating,Bowling,Business Center,Business Center,CCTV,Camel Ride,Campfire / Bon Fire,Car Rental,Casino,Catering,Cell Phone Rental,Children's Park,City Shuttle Service,Clubhouse,Coffee Shop / Cafe,Computer Rental,Concierge,Conference Facility,Conference Facility,Courier Service,Courtesy Coach,Currency Exchange,Dance Performances (on demand),Disabled Friendly Facilities,Disabled Friendly Rooms,Discotheque,Doctor on Call,Doorman,Dry Cleaning,Electronic / Magnetic Keys,Electronic Check-Out,Exhibit Space,Express Check-In / Check-Out,Express Laundry,Extra Person / Child - Share the same room,Family Room,Fax Machine,Fire Exit Signs,Fire Place Available,Fire Safety,First-Aid Kit at Front Desk,Fishing,Fitness Equipment,Floor Butler,Flower Shop,Food Facility,Free Internet Access,Free Local Calls,Front Desk,Full Time Operation for All 7 Days,Fun Floats,Games Zone,Gardens,Gift Shop,Golf Course,Grocery,Guide / Sightseeing Service,Health Club / Gym Facility Available,Hookah Lounge,Horse Ride ( Chargeable ),Hotel Trading License,In House Events,Internet / Fax (Reception area only),Internet Access,Internet Access,Internet Access - Surcharge,Internet Access Free,Jacuzzi,Karaoke,Karaoke,Kids Play Zone,Kitchen available (home cook food on request),Laptop Available on Hire,Laundry Service Available,Library,Lift / Elevator,Limousine Service Available,Lobby,Local Tour / Travel Desk,Locker Facility,Luggage Storage,Major Credit Cards Accepted,Marina On Site,Massage Services,Massage Services,Medical Assistance Available,Meditation Room,Metal Detectors,Mini Theatre,Multi - Line Phone,Multi Cuisine Restaurant,Multi Cuisine Restaurant,Multi Lingual Staff,Nature Walk,Newspapers In Lobby,Night Club,Non Smoking Hotel,Non Smoking Rooms,Open Air Restaurant / Dining,Open Air Restaurant / Dining,Outdoor Parking - Non Secured,Outdoor Parking - Secured,Oxygen Cylinder (on request),Paid Transfers,Parking (Surcharge),Parking Facilities Available,Parking Facility,Party hall,Pest Control Facility,Pets Allowed,"Pick Up and Drop (Rly Station, Bus Stand, Temple)",Picnic Area,Pool Cafe,Pool Table,Poolside Bar,Porter / Bellhop,Porter Services Available,Postal / Parcel Services,Power Backup,Private / Plunge Pool,Private Beach,Public Restrooms,RO Water Purification System,Recreation Zone,Restaurant,Rooftop Garden,Room Service,Room Service,Room Service (24 Hours),STD / ISD,Safari,Sauna,Secretarial Services,Security at All Hotel Entrances,Self Check - In,Separate Sitting Area,Shopping Drop Facility (on fixed timings),Ski Storage,Skiing Facility Available,Smoke Detector,Smoking Area,Smoking Area,Smoking Rooms,Solarium,Spa On Site - Chargeable,Spa Services Nearby,Spa Services Onsite - Free,Speciality Restaurant,Speciality Restaurant,Squash court,Stamps and Mailing Facility,Suitable For Children,Suites,Swimming Pool,Table Tennis,Taxi Services,Tennis Court,Transfers Available,Translation Services,Trekking,Valet service,Veg / Non Veg Kitchens Separate,Vegetarian Food / Jain Food Available,Veranda,Village Visit ( Chargeable ),Virtual Golf,Voicemail,Wake-up Call / Service,Water Dispenser Available,Wedding Services Facility,Welcome Drinks,Wheelchair Accessibility - Room,Whirlpool Bath / Shower Cubicle,X-Ray Machine at Entrance,Yoga Hall / Classes,Hot / Cold Running Water,120 AC,220 AC,Air Conditioning,Air Conditioning,Air Cooler,Alarm Clock,Attached Bathroom,Balcony,Balcony / Terrace,Basic Bathroom Amenities,Bath Tub,

In [62]:
# one-hot property type from df_hotel
from sklearn.preprocessing import LabelBinarizer
# one-hot encoding hotel facilities
lb = LabelBinarizer()
x = lb.fit_transform(df_hotel['property_type'])
df_property_type = pd.DataFrame(x, columns=lb.classes_)

# insert one-hotted property type between site_review_rating and hotel facilities
df_ML_hotel1 = df_ML_hotel.iloc[:,:5]
df_ML_hotel2 = df_ML_hotel.iloc[:,5:]

df_ML_hotel3 =  pd.concat([df_ML_hotel1, df_property_type, df_ML_hotel2], axis=1)

In [63]:
df_ML_hotel3

,hotel_id,latitude,longitude,hotel_star_rating,site_review_rating,Beach Hut,BnB,Bungalow,Cottage,Farm Stay,Guest House,Homestay,Hostel,Hotel,Houseboat,Lodge,Motel,Palace,Resort,Service Apartment,Tent,Villa,Bike on Rent,Cook Services,Drivers Rest Room,24 Hour Front Desk,ATM / Banking,Access Via Exterior Corridors,Adjoining Rooms,Air / Rail Booking,Air-Conditioning - Central,Airport Transfer Available / Surcharge,Airport Transfer Free,Amphitheatre,Anti-Slip Ramps,Audio - Visual Equipment,Ayurvedic Facilities,Ayurvedic Facilities,Babysitting/ Child Care (surcharge),Badminttion Court,Baggage Room,Ballroom,Banquet Facilities,Bar / Lounge,Barbecue Area,Basketball Court,Beauty Salon - on charge,Billiards,Board Room,Boating,Bowling,Business Center,Business Center,CCTV,Camel Ride,Campfire / Bon Fire,Car Rental,Casino,Catering,Cell Phone Rental,Children's Park,City Shuttle Service,Clubhouse,Coffee Shop / Cafe,Computer Rental,Concierge,Conference Facility,Conference Facility,Courier Service,Courtesy Coach,Currency Exchange,Dance Performances (on demand),Disabled Friendly Facilities,Disabled Friendly Rooms,Discotheque,Doctor on Call,Doorman,Dry Cleaning,Electronic / Magnetic Keys,Electronic Check-Out,Exhibit Space,Express Check-In / Check-Out,Express Laundry,Extra Person / Child - Share the same room,Family Room,Fax Machine,Fire Exit Signs,Fire Place Available,Fire Safety,First-Aid Kit at Front Desk,Fishing,Fitness Equipment,Floor Butler,Flower Shop,Food Facility,Free Internet Access,Free Local Calls,Front Desk,Full Time Operation for All 7 Days,Fun Floats,Games Zone,Gardens,Gift Shop,Golf Course,Grocery,Guide / Sightseeing Service,Health Club / Gym Facility Available,Hookah Lounge,Horse Ride ( Chargeable ),Hotel Trading License,In House Events,Internet / Fax (Reception area only),Internet Access,Internet Access,Internet Access - Surcharge,Internet Access Free,Jacuzzi,Karaoke,Karaoke,Kids Play Zone,Kitchen available (home cook food on request),Laptop Available on Hire,Laundry Service Available,Library,Lift / Elevator,Limousine Service Available,Lobby,Local Tour / Travel Desk,Locker Facility,Luggage Storage,Major Credit Cards Accepted,Marina On Site,Massage Services,Massage Services,Medical Assistance Available,Meditation Room,Metal Detectors,Mini Theatre,Multi - Line Phone,Multi Cuisine Restaurant,Multi Cuisine Restaurant,Multi Lingual Staff,Nature Walk,Newspapers In Lobby,Night Club,Non Smoking Hotel,Non Smoking Rooms,Open Air Restaurant / Dining,Open Air Restaurant / Dining,Outdoor Parking - Non Secured,Outdoor Parking - Secured,Oxygen Cylinder (on request),Paid Transfers,Parking (Surcharge),Parking Facilities Available,Parking Facility,Party hall,Pest Control Facility,Pets Allowed,"Pick Up and Drop (Rly Station, Bus Stand, Temple)",Picnic Area,Pool Cafe,Pool Table,Poolside Bar,Porter / Bellhop,Porter Services Available,Postal / Parcel Services,Power Backup,Private / Plunge Pool,Private Beach,Public Restrooms,RO Water Purification System,Recreation Zone,Restaurant,Rooftop Garden,Room Service,Room Service,Room Service (24 Hours),STD / ISD,Safari,Sauna,Secretarial Services,Security at All Hotel Entrances,Self Check - In,Separate Sitting Area,Shopping Drop Facility (on fixed timings),Ski Storage,Skiing Facility Available,Smoke Detector,Smoking Area,Smoking Area,Smoking Rooms,Solarium,Spa On Site - Chargeable,Spa Services Nearby,Spa Services Onsite - Free,Speciality Restaurant,Speciality Restaurant,Squash court,Stamps and Mailing Facility,Suitable For Children,Suites,Swimming Pool,Table Tennis,Taxi Services,Tennis Court,Transfers Available,Translation Services,Trekking,Valet service,Veg / Non Veg Kitchens Separate,Vegetarian Food / Jain Food Available,Veranda,Village Visit ( Chargeable ),Virtual Golf,Voicemail,Wake-up Call / Service,Water Dispenser Available,Wedding Services Facility,Welcome Drinks,Wheelchair Accessibility - Room,Whirlpool Bath / Shower Cubicle,X-Ray Machine at Entrance,Yoga Hall / Classes,Hot / Cold Running Water,120 AC,22

In [ ]:
df_user_final = df_user.copy()
for col in p_hotel2.columns:
    p_user2[col] = 0

p_user2['hotel_id'] = np.empty((len(p_user2), 0)).tolist()
p_user2